In [1]:
%matplotlib widget

In [1]:
import cv2 as cv
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
import decimal
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from mpl_toolkits.mplot3d import Axes3D
from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['Microsoft YaHei']
#改成先segmentation再做計算

#parameters
BAS = 20.0 #cm
SIG = 0.001
AMOUNT = 9 #amount of validate images
TE = 9 #amount of calibration images for each distance

#camera parameters
F = 1.6 #cm 1.6 for sony ,  0.15 for nyuV2
D1 = F/2.8 #cm   2.8 for T, 5 for F
D2 = F/5.6 #cm   5.6 for T, 10 for F
S =  (BAS*F)/(BAS-F) #cm when focus is at 20cm
PTM = 0.007257 #pixel to cm
M2 = -((D2*S)/np.sqrt(8))
C2 = (-M2)*((1/F)-(1/S))
PAR = (D1/D2)**2
KER = 48
IMG_SIZE = (1440,720)
accept = np.linspace(0.0001,((M2/1000)+C2+0.001),10000)
segment = None

def mouse_callback(event, x, y, flags, param):
    global D
    global O2
    if event == cv.EVENT_LBUTTONDOWN:
        print(x," Y:", y)
        print('d value:', D[y, x])
        #print('o changed value:', O2[y, x])
        #print('o2 value:', o_print[y//window_size, x//window_size])
        #cv.imwrite("compare.jpg",d)
        #print('d2 value:', d2[y, x])
        #print('o2 value:', temp[y, x])

def alignment(img1,img2): #fix it
    im1 = cv.cvtColor(img1, cv.COLOR_BGR2RGB)
    im2 = cv.cvtColor(img2, cv.COLOR_BGR2RGB)

    im1_gray = cv.cvtColor(im1, cv.COLOR_BGR2GRAY)
    im2_gray = cv.cvtColor(im2, cv.COLOR_BGR2GRAY)

    MAX_NUM_FEATURES = 10000
    orb = cv.ORB_create(MAX_NUM_FEATURES)
    keypoints1, descriptors1 = orb.detectAndCompute(im1_gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(im2_gray, None)

    matcher = cv.DescriptorMatcher_create(cv.DescriptorMatcher_BRUTEFORCE_HAMMING)
    matches = matcher.match(descriptors1, descriptors2, None)

    matches = sorted(matches,key = lambda x:x.distance)

    num_good_matches = int(len(matches) * 0.5)
    matches = matches[:num_good_matches]

    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt
    try:
        retval = cv.findHomography(points2, points1, cv.RANSAC)
    except:
        return img2
    h, mask = retval[0], retval[1]

    height, width, channels = im1.shape
    try:
        im2_warp = cv.warpPerspective(im2, h, (width, height))
        im2_warp = cv.cvtColor(im2_warp, cv.COLOR_BGR2RGB)
        return im2_warp
    except:
        return img2

def gaussian_pdf(x, mu, sigma):
    prob = 1/np.sqrt(2*np.pi)/sigma*np.exp(-(x-mu)**2/2/sigma**2)
    return prob
def parzen_window_pdf(x, data, sigma):
    px = [gaussian_pdf(x, mu=mu, sigma=sigma) for mu in data]
    return np.mean(np.array(px), axis=0)
def drange(x, y, jump):
  while x < y:
    yield float(x)
    x += decimal.Decimal(jump)
CLAHE = cv.createCLAHE(clipLimit=1.0, tileGridSize=(16,16))
BRIGHTNESS = 136.0/255.0 #not visible
"""pictures for checking which is better"""
def compute(g1,g2,flag = 0):
    global segment
    lower_blue = np.array([80, 50, 50])
    upper_blue = np.array([100, 255, 255])

    #alignment
    g1 = cv.resize(g1,IMG_SIZE ,interpolation=cv.INTER_AREA)  # sony (1440,720) nyuV2: (576,288) 
    g2 = cv.resize(g2,IMG_SIZE ,interpolation=cv.INTER_AREA)

    
    #改成HSV比較好取值
    ori_g = cv.cvtColor(g2,cv.COLOR_BGR2HSV)

    g1 = cv.cvtColor(g1,cv.COLOR_BGR2GRAY)
    g2 = cv.cvtColor(g2,cv.COLOR_BGR2GRAY)

    ori = g2.copy()
    
    #brightness adjustment

    g1 = CLAHE.apply(g1)
    g2 = CLAHE.apply(g2)
    """
    cv.imshow("First",g2)
    cv.imshow("Second",ori_g)
    cv.waitKey(0)
    cv.destroyAllWindows()
    """
    brightnessG1 = np.mean(g1)/255.0
    brightnessG2 = np.mean(g2)/255.0

    alphal = BRIGHTNESS/brightnessG1
    alphar = BRIGHTNESS/brightnessG2
    

    g1 = cv.convertScaleAbs(g1, alpha=alphal, beta=0)
    g2 = cv.convertScaleAbs(g2, alpha=alphar, beta=0)

    g1 = np.float64(g1) / np.float64(255)
    g2 = np.float64(g2) / np.float64(255)

    #cv.imshow("First",g1)
    #cv.imshow("Second",g2)




    vector = cv.getGaussianKernel(5,0.5)
    kernel = vector @ vector.T
    Sg1 = cv.filter2D(g1,-1,kernel)
    Sg2 = cv.filter2D(g2,-1,kernel)
    k_size = 5
    Lg1 = cv.Laplacian(g1,cv.CV_64F,ksize=k_size) #chang to g2 , original is g1
    dif = (Sg1-Sg2)**2
    Lg = Lg1**2

    if flag == 1:
        return dif,Lg

    mask = cv.inRange(ori_g, lower_blue, upper_blue)
    mask = cv.erode(mask, None, iterations=2) #減少雜訊
    blue_region = cv.bitwise_and(ori_g, ori_g, mask=mask)
    cor = cv.findNonZero(mask)
    if cor is not None:
        cor = np.squeeze(cor)
        top , bottom , left , right = np.min(cor[:,1]),np.max(cor[:,1]),np.min(cor[:,0]),np.max(cor[:,0]) 

    #show = cv.normalize(blue_region, None, alpha=0, beta=255, norm_type=cv.NORM_MINMAX, dtype=cv.CV_8U)#blue_region
    #cv.rectangle(show,(left,bottom),(right,top),(255,255,255),1)
    #cv.imshow("blue",show) #確認用

    #計算中間物的平均值
    #cv.imshow("Second",show)
    cv.waitKey(0)
    cv.destroyAllWindows()
    start_row,end_row = top,bottom
    start_col,end_col = left,right
    seg= np.zeros_like(dif)
    seg[start_row:end_row,start_col:end_col] = 1
    seg = seg.astype(bool)
    return dif,Lg,seg


def segment_cal(dif,Lg):
    window_shape = (KER,KER)
    strides = dif.strides[0]*window_shape[0], dif.strides[1]*window_shape[1], dif.strides[0], dif.strides[1]
    # 創建滑動窗口
    sliding_window = np.lib.stride_tricks.as_strided(dif, shape=(dif.shape[0]//window_shape[0], dif.shape[1]//window_shape[1])+window_shape, strides=strides)
    # 計算滑動窗口內所有元素的和
    sum_dif = np.nansum(sliding_window,axis=(-2,-1),keepdims=True)
    sum_dif = np.squeeze(sum_dif)
    strides = Lg.strides[0]*window_shape[0], Lg.strides[1]*window_shape[1], Lg.strides[0], Lg.strides[1]

    sliding_window = np.lib.stride_tricks.as_strided(Lg, shape=(Lg.shape[0]//window_shape[0], Lg.shape[1]//window_shape[1])+window_shape, strides=strides)
    sum_L1 = np.nansum(sliding_window,axis=(-2,-1),keepdims=True)
    sum_L1 =  np.squeeze(sum_L1)

    G = np.sqrt(16*np.divide(sum_dif,sum_L1))
    G[np.isnan(G)] = np.inf
    o2 = np.sqrt(G/(PAR-1))*PTM
    return o2

def smooth(range,sigma):
    global accept
    flatten = range.flatten()
    prob = parzen_window_pdf(accept,flatten,sigma)
    smooth = accept[np.argmax(prob)]
    return smooth

def calibrate():  
    head = os.listdir("./T")
    l_place = ["./T/"+i+"/L" for i in head]
    r_place = ["./T/"+i+"/R" for i in head]
    seg_mask = [np.load(f"./F_mask/T{i}.npy") for i in range(1,11)]
    distance = [BAS+10*i for i in range(10)]


    fileL = [os.listdir(place) for place in l_place]
    fileR = [os.listdir(place) for place in r_place]

    R = [[cv.imread(top+"/"+file) for file in img[0:3]] for img,top in  zip(fileR,r_place)]
    L = [[cv.imread(top+"/"+file) for file in img[0:3]] for img,top in  zip(fileL,l_place)]
    #index = 4
    #R = [[cv.imread(r_place[index]+"/"+file) for file in fileR[index]]]
    #L = [[cv.imread(l_place[index]+"/"+file) for file in fileL[index]]]
    d = []
    o = []
    o_ori = []
    for r,l,seg_ in zip(R,L,seg_mask):
        imagesR = r
        imagesL = l

        for g1 in imagesR:
            for g2 in imagesL:
                dif,Lg,seg = compute(g1,g2)#改成先segmentation再做計算
                range_o2 = segment_cal(dif,Lg)
                range_o2 = np.repeat(np.repeat(range_o2,KER,axis=0),KER,axis=1)
                result = np.zeros_like(dif)
                for i in range(0,IMG_SIZE[1],5):
                    for j in range(0,IMG_SIZE[0],5):
                        window = range_o2[i:i+5,j:j+5]
                        smoothed = smooth(window,SIG)
                        result[i:i+5,j:j+5] = smoothed
                range_o2 = result[seg]
                range_o2 = np.unique(range_o2)
                smooth_o2 = smooth(range_o2,SIG)
                smooth_d2 = np.abs(M2/(smooth_o2-C2))
                d.append(smooth_d2)
                o.append(smooth_o2)
                print("done")
    dictionary = dict()
    res = list()
    real = list()
    for i in distance:
        for j in range(TE):
            real.append(i)
    dictionary2 = {o[i]:real[i] for i in range(len(o))}
    index = 0
    for _ in range(10):
        res.append(o[index:(index+TE)])
        index += TE
    for dist,part_o in zip(distance,res):
        print(part_o)
        o_temp = np.array(part_o,dtype=np.float64)
        sm = smooth(o_temp,(SIG))
        dictionary.update({sm:dist})
    print(dictionary)
    compute_model(dictionary,dictionary2)
    d = np.array(d)
    plt.figure(figsize=(14,5))
    plt.subplot(1,2,1)
    plt.scatter(range(len(d)),d,label="估計的距離")
    plt.plot(real,label="實際距離")
    plt.title("距離(cm)")
    plt.xlabel("資料ID")
    plt.ylabel("距離(cm)")
    plt.legend()
    plt.subplot(1,2,2)
    plt.plot(o)
    plt.xlabel("資料ID")
    plt.ylabel("模糊度")
    for x in range(0,len(o),TE):
        plt.axvline(x,color='r',linestyle='--')

    plt.title("模糊度")
    
    for i,data in enumerate(res):
        print(f"DIS{distance[i]} : {np.std(data)}")
    """"""
    plt.show()

def model(x):
    y = 1.44494148e+08*x**2 -6.49333857e+04*x + 2.85326365e+01
    #calibrate的原值:1.401e+08*x**2 - 2.179e+04*x + 18.56 good
    #calibrate的原值:7.152e+07*x**2 + 3.508e+04*x + 8.522 good
    #calibrate的原值:1.185e+05*x - 11.77
    #1.143e+08*x**2 - 1.024e+04*x + 17.85
    #1.213e+05*x - 13.55
    #1.492e+10*x**3 + 8.956e+07*x**2 + 2012*x + 16.15
    return y

def f(x,a,b):
    return a*x**2 + b*x

def compute_model(table,table2):
    para,_ = curve_fit(f, list(table.keys()),list(table.values()))
    #models = np.poly1d(np.polyfit(list(table.keys()),list(table.values()),2))
    polyline = np.linspace(0,0.007,1000)
    plt.scatter(list(table2.keys()),list(table2.values()),color='g',label="資料",alpha=0.1)
    plt.scatter(x=list(table.keys()),y=list(table.values()),color='r',label="平滑後的資料")
    plt.plot(polyline,f(polyline,*para),label="二次曲線")
    plt.title("模糊度與實際距離的關係")
    plt.xlabel("模糊度")
    plt.ylabel("實際距離(cm)")
    plt.legend()
    plt.show()
    print(para)

DIFFS = np.zeros((IMG_SIZE[1],IMG_SIZE[0]))
DIFFS_2 = np.zeros((IMG_SIZE[1],IMG_SIZE[0]))
LGS = np.zeros((IMG_SIZE[1],IMG_SIZE[0]))
LGS_2 = np.zeros((IMG_SIZE[1],IMG_SIZE[0]))
DIFF = list()
LG = list()
RESULT = "./result/STMAP/"
def main(s,l,flag_2 = 0,seg_mask = None,idx = 0):
    global segment,DIFF,DIFFS,DIFFS_2,LG,LGS,LGS_2
    # new : 2.62013465e+07 1.57096449e+04 |

    #with brightness adjustment : 
    para = [9.41828623e+06 , -3.60316052e+03] 
    global D
    global O
    global O2   
    #    dfd_dataset/R/2018-04-26T17_26_34.505212.JPG
    # ./T/T03/R/DSC06006.JPG
    #    "./betch/T01/smallD/DSC06169.JPG"
    g1 = cv.imread( s)
    g2 = cv.imread( l)
    if idx !=0:
        error_file = open(f"{RESULT}T{idx}.txt","+a")
    tt = cv.getTickCount()
    dif,Lg = compute(g1,g2,flag=1)
    range_o2 = segment_cal(dif,Lg)
    range_o2 = np.repeat(np.repeat(range_o2,KER,axis=0),KER,axis=1)
    distance = np.zeros_like(dif)
    for i in range(0,IMG_SIZE[1],5):
        for j in range(0,IMG_SIZE[0],5):
                window = range_o2[i:i+5,j:j+5]
                smoothed = smooth(window,SIG)
                distance[i:i+5,j:j+5] = f(smoothed,*para)
    done_time = ((cv.getTickCount() - tt) / cv.getTickFrequency())
    error_file.write(f"{done_time},")
    unique_val = sorted(np.unique(seg_mask))
    error_rates = list()
    for val in unique_val:
        if val == 0:
            continue
        m1 = np.where((seg_mask == val),True,False)
        position = np.where(m1==True)
        dis_temp = distance[position]
        error_rate = (np.mean(dis_temp) - val)/val
        
        error_file.write(f"{error_rate},")
        print(error_rate," ",val," Mean",np.mean(dis_temp))
        error_rate = np.abs(error_rate)
        error_rates.append(error_rate)
    mean = np.mean(error_rates)
    print("Error Rate:",mean)
    error_file.write(f"{mean}\n")
    error_file.close()

    D = distance
    
    """
    mask = (dis>200.0)
    dis[mask] = 200.0
    dis = cv.normalize(dis, None, alpha=0, beta=255, norm_type=cv.NORM_MINMAX, dtype=cv.CV_8U)
    """
    """
    show = cv.normalize(D, None, alpha=0, beta=255, norm_type=cv.NORM_MINMAX, dtype=cv.CV_8U)
    #print(show.max()," ",show.min()," ",D.max()," ",D.min())
    
    
    
    cv.imshow("T",show)
    cv.setMouseCallback('T', mouse_callback)
    cv.waitKey(0)
    cv.destroyAllWindows()
    """
    


In [2]:
#calibrate()

HEAD = "./bench/"
LARGER = "/largerD"
SMALLER = "/smallD"
position = [14]
head = os.listdir(HEAD)
l_place = [HEAD+str(head[i])+LARGER for i in position]
r_place = [HEAD+str(head[i])+SMALLER for i in position]
fileL = [os.listdir(place) for place in l_place]
fileR = [os.listdir(place) for place in r_place]
R = [[top+"/"+file for file in img] for img,top in  zip(fileR,r_place)]
L = [[top+"/"+file for file in img] for img,top in  zip(fileL,l_place)]
MASK = [np.load(f"./T{(i+1)}.npy") for i in position]
def get_shape():
    for i,imgs in enumerate(zip(R,L)):
        r = imgs[0]
        l = imgs[1]
        main(r[0],l[0],(position+i+1),0)
def exp():       
    for i,data in enumerate(zip(R,L,MASK)):
        count = 0
        imagesR = data[0]
        imagesL = data[1]
        idx = position[i]+1
        for g1,g2 in zip(imagesR,imagesL):
                main(g1,g2,0,data[2],idx)
                count += 1
    
#get_shape()
exp()
#devi(100)
D = None
O = NoneO2 = None

#test()

0.07984220482735699   73.7  Mean 79.58437049577621
Error Rate: 0.07984220482735699
0.06271747771933892   73.7  Mean 78.32227810791528
Error Rate: 0.06271747771933892
0.07278046762666365   73.7  Mean 79.06392046408511
Error Rate: 0.07278046762666365
0.0797250974144758   73.7  Mean 79.57573967944687
Error Rate: 0.0797250974144758
0.09670899275758943   73.7  Mean 80.82745276623434
Error Rate: 0.09670899275758943
0.1215065942095111   102.2  Mean 114.61797392821204
Error Rate: 0.1215065942095111
0.15333740128759862   102.2  Mean 117.87108241159258
Error Rate: 0.15333740128759862
0.13851978771637066   102.2  Mean 116.35672230461309
Error Rate: 0.13851978771637066
0.12914703080812792   102.2  Mean 115.39882654859068
Error Rate: 0.12914703080812792
0.14707920645162514   102.2  Mean 117.2314948993561
Error Rate: 0.14707920645162514
-0.5844508361064399   21.5  Mean 8.934307023711542
0.22994504353151096   62.9  Mean 77.36354323813204
-0.0076275530440801775   102.2  Mean 101.42046407889501
Error R